# 캐글 데이터 다운로드 받는 법
- 캐글에서 받은 API key 필요

In [ ]:
from google.colab import drive

drive.mount('/gdrive')


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [16]:
import os, json

with open('kaggle.json') as f:
    kaggle = json.load(f)
os.environ['KAGGLE_USERNAME']= kaggle['username']
os.environ['KAGGLE_KEY']=kaggle['key']


In [17]:
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset


microsoft-catsvsdogs-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip microsoft-catsvsdogs-dataset.zip > /dev/null

replace MSR-LA - 3467.docx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [18]:
cat_dir= 'PetImages/Cat'
os.unlink(f'{cat_dir}/Thumbs.db')
cat_list = os.listdir(cat_dir)
len(cat_list)

12500

In [19]:
dog_dir= 'PetImages/Dog'
os.unlink(f'{dog_dir}/Thumbs.db')
dog_list = os.listdir(dog_dir)
len(dog_list)

12500

미지 파일 에러 찾기

In [20]:
from PIL import Image
from glob import glob

In [23]:
for fname in cat_list:
    try:
        img=Image.open(f'PetImages/Cat/{fname}')
    except:
        print(fname)        

666.jpg


In [25]:
for fname in dog_list:
    try:
        img=Image.open(f'PetImages/Dog/{fname}')
    except:
        print(fname)        

11702.jpg


/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

In [26]:
!cp PetImages/Cat/10000.jpg PetImages/Cat/666.jpg
!cp PetImages/Dog/10000.jpg PetImages/Dog/11702.jpg

- 0~7999.jpg=train
- 8000~9999 = valid
- 10000~12499= test

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
seed=2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [28]:
from PIL import Image
IMAGE_SIZE=150
def center_image(img, src_format='OpenCV', dst_format='OpenCV'):
    if src_format == 'OpenCV':
        h,w = img.shape[:-1]
    else:
        h,w = np.array(img).shape[:-1]
    
    if h > w:
        width, height = IMAGE_SIZE, (h * IMAGE_SIZE) // w
    else:
        width, height = (w*IMAGE_SIZE) // h, IMAGE_SIZE
    interpolation=cv2.INTER_AREA if h + w > 300 else cv2.INTER_CUBIC
    new_img=cv2.resize(img,dsize=(width,height),interpolation=interpolation)

    diff=abs(width-height) //2
    if h > w:
        final_img=new_img[diff:diff+IMAGE_SIZE, :]
    else:
        final_img=new_img[:,diff:diff+IMAGE_SIZE]

    return final_img if dst_format =='OpenCV' else Image.fromarray(final_img)


In [29]:
CENTER_DIR = 'cats_and_dogs_centered'
train_dir = f'{CENTER_DIR}/train'
validation_dir = f'{CENTER_DIR}/validation'
train_cats_dir = f'{CENTER_DIR}/train/cats'
train_dogs_dir = f'{CENTER_DIR}/train/dogs'
validation_cats_dir = f'{CENTER_DIR}/validation/cats'
validation_dogs_dir = f'{CENTER_DIR}/validation/dogs'


In [30]:
import os
if not os.path.exists(CENTER_DIR):
    os.mkdir(CENTER_DIR)
for dir in [train_dir,validation_dir,train_cats_dir,train_dogs_dir,validation_cats_dir,validation_dogs_dir]:
    os.mkdir(dir)


In [32]:
old_train_cats_dir = 'cats_and_dogs_filtered/train/cats'
old_train_dogs_dir = 'cats_and_dogs_filtered/train/dogs'
old_validation_cats_dir = 'cats_and_dogs_filtered/validation/cats'
old_validation_dogs_dir = 'cats_and_dogs_filtered/validation/dogs'


In [ ]:
train_dir = 'cats_and_dogs_filtered/train'
validation_dir = 'cats_and_dogs_filtered/validation'
train_cats_dir = 'cats_and_dogs_filtered/train/cats'
train_dogs_dir = 'cats_and_dogs_filtered/train/dogs'
validation_cats_dir = 'cats_and_dogs_filtered/validation/cats'
validation_dogs_dir = 'cats_and_dogs_filtered/validation/dogs'


In [ ]:
# cats_and_dogs_filtered -> image 축소/확대 후 센터링 -> cats_and_dogs_centered 로 보냄.
from glob import glob

for src_dir in [old_train_cats_dir, old_train_dogs_dir,old_validation_cats_dir,old_validation_dogs_dir]:
#    dst_dir = src_dir.split('/')[1:]
#   dst_dir = '/'.join([CENTER_DIR]+dst_dir)
    dst_dir=src_dir.replace('filtered','centered')

    for fname in glob(f'{src_dir}/*.jpg'):
        dst_fname = fname.split('/')[-1]
        src_img = cv2.imread(fname)[:,:,::-1]
        dst_img = center_image(src_img, src_format='OpenCV',dst_format='Pillow')
        dst_img.save(f'{dst_dir}/{dst_fname}')

